In [3]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, TobaccoUse
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
from functools import reduce



#Gender und Race getrennt, Age in Stufen angeben (wie bei has condition) ? 

In [4]:
#load normal Cohorts
Cohort = pq.read_table('Cohorts/ML/Ready_to_train_BMI_BP_Pulse_4CV_5Drugs_wo_NaN.parquet').to_pandas()

In [5]:
Case = Cohort.loc[Cohort["HT"]==1]

In [6]:
Control = Cohort.loc[Cohort["HT"]==0]


In [93]:
Case.head()



,Age_at_prediction,BMI_max,BMI_mean,BMI_min,DBP_max,DBP_mean,DBP_min,HT,SBP_max,SBP_mean,...,has_condition_veins_and_lyphatics,number_of_occurences_Med_CV,number_of_occurences_Med_depression,number_of_occurences_Med_lipid_disorders,number_of_occurences_Med_schizo,number_of_occurences_Med_type2_diabetes,pulse_rate_max,pulse_rate_mean,pulse_rate_min,age_in_days
medical_record_number,,,,,,,,,,,,,,,,,,,,,
1000012395,11362.0,34.739230,31.984547,30.618890,89.0,82.500000,76.0,1,152.0,139.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,71.444444,58.0,37
1000227914,20937.0,22.670535,22.311820,21.953105,98.0,83.666667,70.0,1,150.0,126.666667,...,1.0,0.0,3.0,0.0,0.0,0.0,97.0,85.666667,76.0,68
1000508752,25032.0,23.504128,22.335818,21.273691,91.0,69.052632,54.0,1,151.0,127.631579,...,0.0,0.0,0.0,0.0,0.0,0.0,101.0,83.615385,65.0,75
1000519317,29647.0,38.291528,38.291528,38.291528,84.0,84.000000,84.0,1,120.0,120.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,82.0,71.800000,64.0,91
1000560997,14034.0,27.983299,26.437846,25.683551,94.0,92.500000,91.0,1,144.0,138.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,102.0,82.000000,65.0,43


In [94]:
for col in Case.columns: 
    print(col)

Age_at_prediction
BMI_max
BMI_mean
BMI_min
DBP_max
DBP_mean
DBP_min
HT
SBP_max
SBP_mean
SBP_min
gender_Female
gender_Male
has_condition_Med_CV
has_condition_Med_depression
has_condition_Med_lipid_disorders
has_condition_Med_schizo
has_condition_Med_type2_diabetes
has_condition_cr
has_condition_ihd
has_condition_veins_and_lyphatics
number_of_occurences_Med_CV
number_of_occurences_Med_depression
number_of_occurences_Med_lipid_disorders
number_of_occurences_Med_schizo
number_of_occurences_Med_type2_diabetes
pulse_rate_max
pulse_rate_mean
pulse_rate_min
age_in_days


In [8]:
#avarage age Case 
Case_av_age= ((Case["Age_at_prediction"].mean()/356))
Case_av_age

55.71798688083756

In [9]:
#avarage age Control 
Control_av_age= ((Control["Age_at_prediction"].mean()/356))
Control_av_age

43.76973612537972

In [10]:
#Standard Deviation Age Control
Control_std_age= ((Control["Age_at_prediction"].std())/356)
Control_std_age

15.871836808488514

In [11]:
#Standard Deviation Age Case
Case_std_age= ((Case["Age_at_prediction"].std())/356)
Case_std_age

15.915886275584171

In [12]:
#count of certain condition 
Case_condition= Case.loc[Case["has_condition_veins_and_lyphatics"]!=0]
print( Case_condition.shape[0])
percentage=Case_condition.shape[0]/Case.shape[0]
print (percentage)

2372
0.09896941628071933


In [85]:
#function for statistics: 
def get_base_characteristic_value(df , characteristic , kind):    
    if kind=="mean": 
        df_mean=df[characteristic].mean()
        df_std= df[characteristic].std()
        df_max= df[characteristic].max()
        df_min= df[characteristic].min()
        base_characteristics_cohort=pd.DataFrame({'Variable': [characteristic+"_mean", characteristic+"_std", characteristic+"_max", characteristic+"_min"],
                                                  'Value': [df_mean, df_std, df_max, df_min],})
       
    if kind=="count": 
        df_condition=df.loc[df[characteristic]!=0]
        df_percent= df_condition.shape[0]/df.shape[0]
        base_characteristics_cohort=pd.DataFrame({'Variable': [characteristic+"_total", characteristic+"_relation"],
                                                  'Value': [df_condition.shape[0], df_percent]})
       # print(df_condition.shape[0], df_percent)
    #print (base_characteristics_cohort)
    return base_characteristics_cohort

In [97]:
base_characteristics=[
    [ "Age_at_prediction","mean"],
    [ "age_in_days","mean"],
    [ "BMI_max","mean"],
    [ "BMI_mean","mean"],
    [ "BMI_min","mean"],
    [ "DBP_max","mean"],
    [ "DBP_mean","mean"],
    [ "DBP_min","mean"],
    [ "SBP_max","mean"],
    [ "SBP_mean","mean"],
    [ "SBP_min","mean"],
    [ "pulse_rate_max","mean"],
    [ "pulse_rate_mean","mean"],
    [ "pulse_rate_min","mean"],
    ["gender_Female" , "count"],
    ["gender_Male" , "count"],
    ["has_condition_Med_CV" , "count"],
    ["has_condition_Med_depression" , "count"],
    ["has_condition_Med_lipid_disorders" , "count"],
    ["has_condition_Med_schizo" , "count"],
    ["has_condition_Med_type2_diabetes" , "count"],
    ["has_condition_cr" , "count"],
    ["has_condition_ihd" , "count"],
    ["has_condition_veins_and_lyphatics" , "count"],
    ["number_of_occurences_Med_CV" , "count"],
    ["number_of_occurences_Med_depression" , "count"],
    ["number_of_occurences_Med_lipid_disorders" , "count"],
    ["number_of_occurences_Med_schizo" , "count"],
    ["number_of_occurences_Med_type2_diabetes" , "count"],
    
    
]

In [105]:
def get_base_characteristics(df, characteristics): 
    base_characteristics_cohort=pd.DataFrame(columns=["Variable","Value"])
    for characteristic in characteristics:
        intermediate_base_characteristics_cohort=get_base_characteristic_value(df,characteristic[0],characteristic[1])
        base_characteristics_cohort=pd.concat([base_characteristics_cohort,intermediate_base_characteristics_cohort])
    print(base_characteristics_cohort)
    return base_characteristics_cohort

In [106]:
Control_baseline=get_base_characteristics(Control, base_characteristics)
Case_baseline=get_base_characteristics(Case, base_characteristics)

                                             Variable         Value
0                              Age_at_prediction_mean  15582.026061
1                               Age_at_prediction_std   5650.373904
2                               Age_at_prediction_max  48668.000000
3                               Age_at_prediction_min   3244.000000
0                                    age_in_days_mean     46.154029
1                                     age_in_days_std     15.335158
2                                     age_in_days_max     92.000000
3                                     age_in_days_min     20.000000
0                                        BMI_max_mean     27.331091
1                                         BMI_max_std      6.005129
2                                         BMI_max_max     99.992422
3                                         BMI_max_min      1.612784
0                                       BMI_mean_mean     25.994969
1                                        BMI_mea

In [110]:
Control_baseline.to_csv('Cohort_Baseline.csv', index = None, header= True)
Case_baseline.to_csv('Case_Baseline.csv', index = None, header= True)

In [107]:
Control_baseline

,Variable,Value
0,Age_at_prediction_mean,15582.026061
1,Age_at_prediction_std,5650.373904
2,Age_at_prediction_max,48668.000000
3,Age_at_prediction_min,3244.000000
0,age_in_days_mean,46.154029
1,age_in_days_std,15.335158
2,age_in_days_max,92.000000
3,age_in_days_min,20.000000
0,BMI_max_mean,27.331091
1,BMI_max_std,6.005129
